In [1]:
from strands import Agent
from strands.tools.mcp import MCPClient
from mcp import stdio_client, StdioServerParameters
import os
import boto3
import subprocess
from utils.ssm import parameter_store

# 환경변수 설정
region = boto3.Session().region_name
pm = parameter_store(region)

opensearch_url = pm.get_params(key="opensearch_domain_endpoint", enc=False)
opensearch_username = pm.get_params(key="opensearch_user_id", enc=False)
opensearch_password = pm.get_params(key="opensearch_user_password", enc=True)

# 환경변수 설정
os.environ["OPENSEARCH_URL"] = opensearch_url
os.environ["OPENSEARCH_USERNAME"] = opensearch_username
os.environ["OPENSEARCH_PASSWORD"] = opensearch_password

# 먼저 mcp_server_opensearch가 실행 가능한지 테스트
try:
    # 서버가 제대로 실행되는지 확인
    test_process = subprocess.Popen(
        ["python", "-m", "mcp_server_opensearch", "--help"],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        env=os.environ.copy()
    )
    stdout, stderr = test_process.communicate(timeout=5)
    
    if test_process.returncode != 0:
        print(f"MCP 서버 테스트 실패:")
        print(f"STDOUT: {stdout.decode()}")
        print(f"STDERR: {stderr.decode()}")
        raise Exception("MCP 서버를 실행할 수 없습니다.")
    
except subprocess.TimeoutExpired:
    test_process.kill()
    print("MCP 서버 테스트 타임아웃 - 서버는 정상적으로 실행될 것 같습니다.")
except Exception as e:
    print(f"MCP 서버 테스트 중 오류: {e}")

# OpenSearch MCP 서버 연결 설정 (환경변수를 명시적으로 전달)
env_vars = os.environ.copy()
env_vars.update({
    "OPENSEARCH_URL": opensearch_url,
    "OPENSEARCH_USERNAME": opensearch_username,
    "OPENSEARCH_PASSWORD": opensearch_password
})

opensearch_mcp_client = MCPClient(
    lambda: stdio_client(StdioServerParameters(
        command="python",
        args=["-m", "mcp_server_opensearch"],
        env=env_vars  # 환경변수 명시적으로 전달
    ))
)

# Agent 생성 및 MCP 도구 연결
try:
    print("MCP 클라이언트 시작 중...")
    with opensearch_mcp_client:
        print("MCP 클라이언트가 성공적으로 시작되었습니다.")
        
        # 사용 가능한 도구 확인
        tools = opensearch_mcp_client.list_tools_sync()
        #print(f"사용 가능한 도구: {[tool.name for tool in tools]}")
        
        # Agent 생성
        agent = Agent(tools=tools)
        
        # Agent 사용
        print("OpenSearch 쿼리 실행 중...")
        response = agent("OpenSearch에서 모든 인덱스를 보여줘")
        print("응답:", response)
        #response = agent("shipment_tracking 인덱스의 데이터는 어떻게 생겼어?")
        response = agent("SearchIndexTool 사용해서 어떤 결과라도 보여줘")
        
        
        print("응답:", response)
        
except Exception as e:
    print(f"MCP 클라이언트 오류: {e}")
    print(f"환경변수 확인:")
    print(f"OPENSEARCH_URL: {os.environ.get('OPENSEARCH_URL', 'NOT SET')}")
    print(f"OPENSEARCH_USERNAME: {os.environ.get('OPENSEARCH_USERNAME', 'NOT SET')}")
    print(f"OPENSEARCH_PASSWORD: {'SET' if os.environ.get('OPENSEARCH_PASSWORD') else 'NOT SET'}")

MCP 클라이언트 시작 중...
MCP 클라이언트가 성공적으로 시작되었습니다.
OpenSearch 쿼리 실행 중...
OpenSearch에서 모든 인덱스 목록을 확인하겠습니다.
Tool #1: ListIndexTool
OpenSearch에 다음과 같은 인덱스들이 존재합니다:

시스템/플러그인 관련 인덱스:
- `.plugins-ml-model-group`
- `.plugins-flow-framework-state`
- `.ql-datasources`
- `.plugins-ml-agent`
- `.plugins-ml-task`
- `.plugins-flow-framework-templates`
- `.opendistro_security`
- `.kibana_1`
- `.opensearch-observability`
- `.plugins-ml-config`
- `.plugins-ml-model`
- `.plugins-flow-framework-config`

비즈니스 데이터 관련 인덱스:
- `inventory_levels`
- `ira_compliance`
- `order_fulfillment`
- `supplier_performance`
- `shipment_tracking`

특정 인덱스에 대해 더 자세한 정보가 필요하시면 말씀해 주세요.응답: OpenSearch에 다음과 같은 인덱스들이 존재합니다:

시스템/플러그인 관련 인덱스:
- `.plugins-ml-model-group`
- `.plugins-flow-framework-state`
- `.ql-datasources`
- `.plugins-ml-agent`
- `.plugins-ml-task`
- `.plugins-flow-framework-templates`
- `.opendistro_security`
- `.kibana_1`
- `.opensearch-observability`
- `.plugins-ml-config`
- `.plugins-ml-model`
- `.plugins-flow-frame

In [2]:
agent("shipment_tracking 인덱스의 데이터는 어떻게 생겼어?")

shipment_tracking 인덱스의 데이터를 확인하겠습니다. 먼저 인덱스의 매핑 정보를 살펴본 후, 샘플 데이터를 검색하겠습니다.
Tool #2: IndexMappingTool
죄송합니다. 인덱스 매핑 정보를 가져오는 도중 오류가 발생했습니다. 대신 shipment_tracking 인덱스에서 샘플 데이터를 직접 검색해보겠습니다.
Tool #3: SearchIndexTool


tool_name=<IndexMappingTool> | failed to process tool
Traceback (most recent call last):
  File "/home/sagemaker-user/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/08_bedrock_manus/use_cases/03_supply_chain_analysis_strands_sdk/setup/.venv/lib/python3.12/site-packages/strands/handlers/tool_handler.py", line 105, in process
    return tool_func.invoke(tool, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/sagemaker-user/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/08_bedrock_manus/use_cases/03_supply_chain_analysis_strands_sdk/setup/.venv/lib/python3.12/site-packages/strands/tools/mcp/mcp_agent_tool.py", line 83, in invoke
    return self.mcp_client.call_tool_sync(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/sagemaker-user/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/08_bedrock_manus/use_cases/03_supply_chain_analysis_strands_sdk/setup/.venv/lib/python3.12/site-packages/strands/tools/mcp/mcp_client.py", line 189, in c

tool_name=<SearchIndexTool> | failed to process tool
Traceback (most recent call last):
  File "/home/sagemaker-user/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/08_bedrock_manus/use_cases/03_supply_chain_analysis_strands_sdk/setup/.venv/lib/python3.12/site-packages/strands/handlers/tool_handler.py", line 105, in process
    return tool_func.invoke(tool, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/sagemaker-user/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/08_bedrock_manus/use_cases/03_supply_chain_analysis_strands_sdk/setup/.venv/lib/python3.12/site-packages/strands/tools/mcp/mcp_agent_tool.py", line 83, in invoke
    return self.mcp_client.call_tool_sync(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/sagemaker-user/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/08_bedrock_manus/use_cases/03_supply_chain_analysis_strands_sdk/setup/.venv/lib/python3.12/site-packages/strands/tools/mcp/mcp_client.py", line 189, in ca

죄송합니다만, 시스템에서 현재 클라이언트 세션이 실행되지 않는 기술적 문제가 발생하고 있습니다. 다시 시도해보겠습니다.

KeyboardInterrupt: 

In [18]:
# Agent 생성 및 MCP 도구 연결
with opensearch_mcp_client:
    agent = Agent(tools=opensearch_mcp_client.list_tools_sync())
    
    # Agent 사용
    response = agent("OpenSearch에서 모든 인덱스를 보여줘")
    print(response)

client failed to initialize
  + Exception Group Traceback (most recent call last):
  |   File "/home/sagemaker-user/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/08_bedrock_manus/use_cases/03_supply_chain_analysis_strands_sdk/setup/.venv/lib/python3.12/site-packages/strands/tools/mcp/mcp_client.py", line 107, in start
  |     self._init_future.result(timeout=30)
  |   File "/opt/conda/lib/python3.12/concurrent/futures/_base.py", line 456, in result
  |     return self.__get_result()
  |            ^^^^^^^^^^^^^^^^^^^
  |   File "/opt/conda/lib/python3.12/concurrent/futures/_base.py", line 401, in __get_result
  |     raise self._exception
  |   File "/home/sagemaker-user/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/08_bedrock_manus/use_cases/03_supply_chain_analysis_strands_sdk/setup/.venv/lib/python3.12/site-packages/strands/tools/mcp/mcp_client.py", line 223, in _async_background_thread
  |     async with self._transport_callable() as (read_stream, write_stre

In [6]:
import os
import boto3
from utils.ssm import parameter_store

In [14]:
import os
import boto3
from utils.ssm import parameter_store

region = boto3.Session().region_name
pm = parameter_store(region)

os.environ["OPENSEARCH_URL"] = pm.get_params(key="opensearch_domain_endpoint", enc=False)
os.environ["OPENSEARCH_USERNAME"] = pm.get_params(key="opensearch_user_id", enc=False)
os.environ["OPENSEARCH_PASSWORD"] = pm.get_params(key="opensearch_user_password", enc=True)

python -m mcp_server_opensearch

SyntaxError: invalid syntax (4151573414.py, line 8)

In [11]:
pm.get_params(key="opensearch_user_password", enc=True)


'MarsEarth1!'

In [9]:
import mcp_server_opensearch

In [13]:
mcp_server_opensearch

TypeError: 'module' object is not callable

In [5]:


# FileScopeMCP 서버에 연결
stdio_mcp_client = MCPClient(lambda: stdio_client(
    StdioServerParameters(
        command="node",
        args=[
            "/home/sagemaker-user/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/08_bedrock_manus/use_cases/0x_code_assistant_strands_sdk/setup/FileScopeMCP/dist/mcp-server.js",
            "--base-dir=/home/sagemaker-user/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/08_bedrock_manus/use_cases/0x_code_assistant_strands_sdk"
        ]
    )
))

In [8]:


# Agent 생성 및 사용
with stdio_mcp_client:
    # MCP 서버에서 도구 목록 가져오기

    
    tools = stdio_mcp_client.list_tools_sync()
    
    # Agent 생성
    agent = Agent(tools=tools)
    
    # FileScopeMCP 도구 사용 예시
    response = agent("/home/sagemaker-user/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/08_bedrock_manus/use_cases/01_deep_research 폴더를 분석할꺼야. 각각의 중요도 알려줘")
    print(response)

요청하신 폴더를 분석하기 위해 먼저 해당 경로의 파일 트리를 생성해야 합니다. 그런 다음 각 파일의 중요도를 확인해 드리겠습니다.

먼저 파일 트리를 생성하겠습니다:
Tool #1: create_file_tree
이제 파일 트리가 생성되었으니 모든 파일과 그 중요도를 확인해 보겠습니다:
Tool #2: list_files
이제 더 정확한 중요도 정보를 위해 중요 파일들을 찾아보겠습니다:
Tool #3: find_important_files
### 01_deep_research 폴더 파일 중요도 분석 결과

분석한 폴더에 있는 주요 파일들의 중요도를 기반으로 정리해 드리겠습니다:

#### 가장 중요한 파일들 (중요도 4):
1. `/app/app.py` - 애플리케이션의 주요 실행 파일, 중요도 4
2. `/src/graph/types.py` - 그래프 관련 타입 정의 파일, 중요도 4

#### 높은 중요도의 파일들 (중요도 3):
여러 프롬프트 정의 파일들 (모두 중요도 3):
- `/src/prompts/browser.md`
- `/src/prompts/clarifier.md`
- `/src/prompts/coder.md`
- `/src/prompts/coordinator.md`
- `/src/prompts/file_manager.md`
- `/src/prompts/human_feedback.md`
- `/src/prompts/planner.md`
- `/src/prompts/reporter.md`
- `/src/prompts/researcher.md`
- `/src/prompts/supervisor.md`

#### 중간 중요도의 파일들 (중요도 2):
- `/main.ipynb` - Jupyter 노트북 메인 파일
- `/main.py` - 메인 실행 파일
- `/README.md` - 프로젝트 설명 파일
- `/src` 디렉토리의 여러 Python 모듈 파일들과 초기화 파일들

#### 낮은 중요도의 파일들 (중요도 1):
- `/CONTRIBUTI

In [ ]:
!pwd

In [ ]:
!ls